In [1]:
from gurobipy import *

## 本研究

In [66]:
# nと初期配置を変更しなければいけない
ship_num=4
stack_num=4
height=4
# time = 10
f=stack_num*height-sum(n)

O1=[i+1 for i in range(ship_num)]

S=[i+1 for i in range(stack_num)]
H=[i+1 for i in range(height)]
P=[i+1 for i in range(ship_num)]

n=[1,2,4,3]
# T=[i+1 for i in range(time)]

# S,H,P,T,n1,O

6

In [52]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
import random
from random import seed
import numpy as np
from scipy.stats import multivariate_normal

np.random.seed(0)

# 期待値と分散共分散行列の準備
mean = np.arange(1,2*ship_num,2)
cov=np.zeros((ship_num,ship_num))
for i in range(ship_num):
  for j in range(ship_num):
    if i==j:
      cov[i][i]=4*np.random.randn()
    elif i>j:
      cov[i][j]=np.random.randn()
      cov[j][i]=cov[i][j]

# numpy を用いた生成
size=10
data_1 = np.random.multivariate_normal(mean, cov, size=size)

# print(data_1.shape)

# print(np.mean(data_1, axis=0))

# print(np.cov(data_1, rowvar=False))

# print(data_1)
O_=np.argsort(data_1)
O_=O_+np.ones((size,ship_num)).astype(int)
O=O_.tolist()

# data_2 = multivariate_normal(mean, cov).rvs(size=200)

# print(data_2.shape)

# print(np.mean(data_2, axis=0))

# print(np.cov(data_2, rowvar=False))

C:\Users\Owner\AppData\Local\Temp\ipykernel_10120\239299481.py:22: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size=size)


In [53]:
a=[]
b={}
for i in O:
  # print(i)
  if not i in a:
    a.append(i)
for j in a:
  b[tuple(j)]=O.count(j)

b

{(1, 3, 2, 4): 3,
 (2, 1, 3, 4): 2,
 (1, 4, 2, 3): 1,
 (1, 2, 3, 4): 1,
 (1, 4, 3, 2): 1,
 (3, 2, 1, 4): 1,
 (2, 3, 1, 4): 1}

In [54]:
m=Model("IPS6")

alpha=m.addVar(vtype="C")

# 変数の定義
x,c,d={},{},{}
for s in S:
  for h in H:
    for p in P:
      # for t in T:
      x[s,h,p]=m.addVar(vtype="B")


# for s in S:
#   for h in H:
#     for p in P:
#       for t in T:
#         w[s,h,p,t]=m.addVar(vtype="C",lb=0)
#         z[s,h,p,t]=m.addVar(vtype="C",lb=0)


for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      c[s,h,i]=m.addVar(vtype="c",lb=0)

for i in range(len(O)):
  d[i]=m.addVar(vtype="C",lb=0)

# 制約条件
# for p in P:
#    for t in range(1,len(T)):
#       m.addConstr(quicksum(w[s,h,p,t] for s in S for h in H)==quicksum(z[s,h,p,t] for s in S for h in H))

# for t in range(1,len(T)):
#    m.addConstr(quicksum(w[s,h,p,t] for s in S for h in H for p in P)<=1)
#    m.addConstr(quicksum(z[s,h,p,t] for s in S for h in H for p in P)<=1)

# for t in range(1,len(T)-1):
#   m.addConstr(quicksum(w[s,h,p,t+1] for s in S for h in H for p in P)<=quicksum(w[s,h,p,t] for s in S for h in H for p in P))

for p in P:
  m.addConstr(quicksum(x[s,h,p] for s in S for h in H)==n[p-1])

for s in S:
  for h in H:
    m.addConstr(quicksum(x[s,h,p] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    m.addConstr(quicksum(x[s,h+1,p] for p in P)<=quicksum(x[s,h,p] for p in P))

# for s in S:
#   for h in H:
#     for p in P:
#       for t in range(1,len(T)):
#         m.addConstr(x[s,h,p,t]+z[s,h,p,t]==x[s,h,p,t+1]+w[s,h,p,t])

# for s in S:
#   for h in range(1,len(H)):
#     for t in range(1,len(T)):
#       m.addConstr(quicksum(x[s,h,p,t] for p in P)+quicksum(x[s,h+1,p,t] for p in P)+quicksum(w[s,h,p,t] for p in P)+quicksum(z[s,h+1,p,t] for p in P)+quicksum(z[s,h,p,t] for p in P)<=2)

# for s in S:
#   for t in range(1,len(T)-1):
#     m.addConstr(quicksum(z[s,h,p,t] for h in H for p in P)+quicksum(w[s,h,p,t+1] for h in H for p in P)<=1)

# for s in S:
#   for p in P:
#     for t in range(1,len(T)-1):
#       m.addConstr(quicksum(w[s,h,p,t] for h in H)+quicksum(z[s,h,p,t+1] for h in H)<=1)

# for s in S:
#   for h in range(1,len(H)):
#     for p in P:
#       m.addConstr(quicksum(x[s,h+1,k,len(T)] for k in range(p,len(P)+1))<=quicksum(x[s,h,k,len(T)] for k in range(p,len(P)+1)))

# for s in S:
#   for h in H:
#     m.addConstr(w[s,h,1,len(T)-1]==0)

# for s in S:
#   for p in range(2,len(P)+1):
#     m.addConstr(w[s,1,p,len(T)-1]==0)

# for s in S:
#   for h in H:
#     for p in P:
#       m.addConstr(z[s,h,p,len(T)]==0)
#       m.addConstr(w[s,h,p,len(T)]==0)

for s in S:
  for h in range(2,len(H)+1):
    for h_ in range(1,h):
      for i,o in enumerate(O):
        for j,p in enumerate(o):
          m.addConstr(c[s,h,i]>=quicksum(x[s,h,k] for k in o[j:])-quicksum(x[s,h_,k] for k in o[j:]))

for i in range(len(O)):
  m.addConstr(d[i]>=quicksum(c[s,h,i] for s in S for h in H if h!=1)-alpha)

In [56]:
beta=0.8
# m.setObjective(alpha+10**(-5)*quicksum(y[s,k,t] for s in S for k in S if s!=k for t in range(1,len(T)))+quicksum(w[i] for i in range(len(O)))/((1-beta)*len(O)))
m.setObjective(alpha+quicksum(d[i] for i in range(len(O)))/((1-beta)*len(O)))

In [57]:
if f>=height:
    m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1002 rows, 195 columns and 6124 nonzeros
Model fingerprint: 0xd3f8491e
Variable types: 131 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [61]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  # for (s,h,p,t) in x:
  #   if t==len(T):
  #     if x[s,h,p,t].X>EPS:
  #       print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X)

  for (s,h,i) in c:
    if c[s,h,i].X>EPS:
      print("c[%2s,%2s,%2s]=%3s  %4s" %(s,h,i,c[s,h,i].X,O[i]))

  result=np.zeros((stack_num,height))
  for (s,h,p) in x:
      if x[s,h,p].X>EPS:
          result[height-h][s-1]=int(p)
  result=result.astype(int)
  print(result)
  print("the objective function", m.objVal)
  print("VaR=",alpha.X)

[[0 0 0 3]
 [1 0 0 3]
 [4 0 2 3]
 [4 4 2 3]]
the objective function 0.0
VaR= 0.0


In [59]:
result=np.zeros((stack_num,height))
for (s,h,p) in x:
    if x[s,h,p].X>EPS:
        result[height-h][s-1]=int(p)
result=result.astype(int)
print(result)

[[0 0 0 3]
 [1 0 0 3]
 [4 0 2 3]
 [4 4 2 3]]


## 先行研究１

In [40]:
# nと初期配置を変更しなければいけない
ship_num=4
stack_num=4
height=4
num=10
# time = 10

O1=[i+1 for i in range(ship_num)]

Q=[i+1 for i in range(stack_num)]
L=[i+1 for i in range(height)]
P=[i+1 for i in range(ship_num)]
I=[i+1 for i in range(num)]
f=stack_num*height-len(I)
gamma=[1,2,2,3,3,3,3,4,4,4]
Gamma=3

# T=[i+1 for i in range(time)]

# S,H,P,T,n1,O

In [41]:
m=Model("BI")

# 変数の定義
alpha,beta={},{}
for i in I:
  for q in Q:
      alpha[i,q]=m.addVar(vtype="B")
      beta[i,q]=m.addVar(vtype="B")
J=[]
for i in I:
  J.append([])
  for j in I:
    if gamma[i-1]<gamma[j-1]:
      if gamma[j-1]-gamma[i-1]<=Gamma:
        J[i-1].append(j)

for q in Q:
  m.addConstr(quicksum((alpha[i,q]+beta[i,q]) for i in I)<=len(L))

for i in I:
  m.addConstr(quicksum((alpha[i,q]+beta[i,q]) for q in Q)==1)

for i in I:
  for j in J[i-1]:
    for q in Q:
      m.addConstr(alpha[i,q]+alpha[j,q]+beta[i,q]<=1)

m.setObjective(quicksum(beta[i,q] for i in I for q in Q))

In [42]:
if f>=height:
    m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 154 rows, 80 columns and 580 nonzeros
Model fingerprint: 0x35619123
Variable types: 0 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 7.0000000
Presolve removed 28 rows and 4 columns
Presolve time: 0.01s
Presolved: 126 rows, 76 columns, 552 nonzeros
Variable types: 0 continuous, 76 integer (76 binary)
Found heuristic solution: objective 3.0000000

Root relaxation: objective 0.000000e+00, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000000    0.00000  0.00%     -    0s


In [43]:
import numpy as np
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  # for (s,h,p,t) in x:
  #   if t==len(T):
  #     if x[s,h,p,t].X>EPS:
  #       print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X)

  for (i,q) in beta:
    if beta[i,q].X>EPS:
      print("beta[%2s,%2s]=%3s" %(i,q,beta[i,q].X))

  EPS=1.e-6
  a=[]
  for q in Q:
      a.append([])
  for (i,q) in alpha:
      if alpha[i,q].X>EPS:
          a[q-1].append(gamma[i-1])
  
  for (i,q) in beta:
      if beta[i,q].X>EPS:
          a[q-1].append(gamma[i-1])

  for q in Q:
      a[q-1]=sorted(a[q-1],reverse=True)

  result=np.zeros((stack_num,height))
  for q in Q:
    for i,r in enumerate(a[q-1]):
      result[height-i-1][q-1]=r
      # print(i,r)

  result=result.astype(int)
  print(result)
  print("the objective function", m.objVal)
#   print("VaR=",alpha.X)

[[3 0 0 0]
 [3 0 4 0]
 [3 2 4 0]
 [3 2 4 1]]
the objective function 0.0


In [133]:
a=[]
for q in Q:
    a.append([])
a

[[], [], [], []]